# Lecture 14 Supplementary Notebook

## DSC 40A, Fall 2024

The following cell sets up the necessary imports – don't worry too much about it.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats("svg")

pd.options.plotting.backend = "plotly"

# DSC 40A preferred styles
pio.templates["dsc40a"] = go.layout.Template(
    layout=dict(
        margin=dict(l=30, r=30, t=30, b=30),
        autosize=True,
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True),
        title=dict(x=0.5, xanchor="center"),
    )
)
pio.templates.default = "simple_white+dsc40a"

from IPython.display import HTML
from ipywidgets import interact, widgets

## Gradient descent

$$f(t) = 5t^4 - t^3 - 5t^2 + 2t - 9$$

In [ ]:
def f(t):
    return 5 * (t**4) - (t**3) - 5 * (t**2) + 2 * t - 9

def df(t):
    return 20 * (t**3) - 3 * (t**2) - 10 * t + 2

def create_tangent_line(t):
    slope = df(t)
    intercept = f(t) - slope * t
    return lambda x: intercept + slope * x

**Note**: This notebook has _lots_ of code, but you're not expected to understand most of it. Instead, most of it is there in order to set up the visualizations.

Run the cell below to see a plot of our function, $f$.

In [ ]:
ts = np.linspace(-1.25, 1.25, 1000)
ys = f(ts)

fig = px.line(x=ts, y=ys)
fig.update_layout(xaxis_title='$t$',
                  yaxis_title='$f(t)$',
                  title='$f(t) = 5t^4 - t^3 - 5t^2 + 2t - 9$',
                  width=800)

Run the cell below to see an interactive visualization, where you can change the value of $t$ and see the line tangent to $f$ that passes through the point $(t, f(t))$.

In [ ]:
# def show_tangent(t0):
def show_tangent(t0):
    tan_fn = create_tangent_line(t0)
    fig2 = go.Figure(fig.data)
    fig2.add_trace(go.Scatter(x=[t0], y=[f(t0)], marker={'color': 'red', 'size': 20}, showlegend=False))
    fig2.add_trace(go.Scatter(x=[-5, 5], y=[tan_fn(-5), tan_fn(5)], line={'color': 'red'}, name='Tangent Line'))
    fig2.update_xaxes(range=[-1.25, 1.25]).update_yaxes(range=[-12, -4])
    fig2.update_layout(title=f'Tangent line to f(t) at t = {round(t0, 2)}<br>Slope of tangent line: {round(df(t0), 5)}', xaxis_title='$t$', yaxis_title='$f(t) = 5t^4 - t^3 - 5t^2 + 2t - 9$', showlegend=False)
    return fig2

interact(show_tangent, t0=(-1.25, 1.25))

Run the cell below and click the **▶️ Start animation** button to see an **animated** version of the previous plot.

In [ ]:
play_button = {'label': '▶️ Start animation', 'method': 'animate', 'args': [None]}

stop_button = dict(label='⏯️ Stop animation', method='animate', visible = True,
            args=[(), {'frame': {'duration': 0, 'redraw': False}, 'mode': 'next', 'fromcurrent': True}])

t_range = np.arange(-1.25, 1.26, 0.1)
anim_fig = go.Figure(
    data=[show_tangent(-1.25).data[0], show_tangent(-1.25).data[1], show_tangent(-1.25).data[2]],
    frames=[
        go.Frame(data=[show_tangent(t).data[0], show_tangent(t).data[1], show_tangent(t).data[2]])
        for t in t_range
    ],
    layout=go.Layout(updatemenus=[dict(
            type="buttons",
            buttons=[play_button, stop_button])])
)
anim_fig.update_xaxes(title='$t$', range=[-1.25, 1.25]).update_yaxes(title='$f(t) = 5t^4 - t^3 - 5t^2 + 2t - 9$', range=[-12, -4])

### Gradient descent update rule

Let's start with an initial guess $t_0 = 0$ and a learning rate $\alpha = 0.01$.

$$t_{i + 1} = t_i - \alpha \frac{df}{dt}(t_i)$$

In [ ]:
t = 0
for i in range(50):
    print(round(t, 4), round(f(t), 4))
    t = t - 0.01 * df(t)

We see that pretty quickly, $t_i$ converges to $-0.727$! What does this look like animated? Run the cell below!

In [ ]:
def minimizing_animation(t0, alpha):
    t = t0
    ts = []
    dfts = []
    for i in range(50):
        ts.append(t)
        dfts.append(df(t))
        t = t - alpha * df(t)

    grad_anim = go.Figure(
        data=[fig.data[0], go.Scatter(x=[ts[0]], y=[f(ts[0])], marker={'size': 20}, showlegend=False)],
        frames=[
            go.Frame(data=[fig.data[0], go.Scatter(x=[ts[i]], y=[f(ts[i])], marker={'size': 20}, showlegend=False)])
            for i in range(50)
        ],
        layout=go.Layout(updatemenus=[dict(
            type="buttons",
            buttons=[play_button, stop_button])],
             title=f'Gradient Descent<br>Initial Guess = {t0}&nbsp;&nbsp;&nbsp;&nbsp;Step Size = {alpha}'))


    return grad_anim

In [ ]:
minimizing_animation(t0=0, alpha=0.01)

What if we start with a different initial guess?

In [ ]:
minimizing_animation(t0=1.1, alpha=0.01)

What if we use a different learning rate?

In [ ]:
minimizing_animation(t0=0, alpha=0.1)

Some learning rates are so large that the values of $t$ explode towards infinity! Watch what happens when we use a learning rate of 1:

In [ ]:
t = 0
for i in range(50):
    print(round(t, 4), round(f(t), 4))
    t = t - 1 * df(t)